In [25]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [26]:

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME","STATUS","SPECIAL_CONSIDERATIONS"],inplace=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [27]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_count = application_df["APPLICATION_TYPE"].value_counts()
app_count

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = app_count.index[8:]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [28]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_count = application_df["CLASSIFICATION"].value_counts()
class_count

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_count[class_count < 10].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: count, dtype: int64

In [30]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df[["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT"]])
application_df.drop(columns=["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT"],inplace=True)
application_dummies_df = pd.merge(application_df, application_dummies_df, how="inner",left_index=True,right_index=True)
application_dummies_df

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
34295,5000,0,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
34296,5000,0,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
34297,5000,1,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False


In [31]:
# Split our preprocessed data into our features and target arrays
y = application_dummies_df['IS_SUCCESSFUL']

# Separate the X variable, the features
X = application_dummies_df.drop(columns=['IS_SUCCESSFUL']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1,train_size=0.85)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# First Attempt at Optimization

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
layer1 = 40
layer2 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1, activation="leaky_relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, activation="leaky_relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

c:\Users\rmt20\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                │ (None, 40)             │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 20)             │           820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,441 (13.44 KB)

 Trainable params: 3,441 (13.44 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 2s 776us/step - accuracy: 0.7029 - loss: 0.6012
Epoch 2/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 770us/step - accuracy: 0.7309 - loss: 0.5563
Epoch 3/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 805us/step - accuracy: 0.7306 - loss: 0.5474
Epoch 4/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 803us/step - accuracy: 0.7348 - loss: 0.5468
Epoch 5/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 837us/step - accuracy: 0.7269 - loss: 0.5514
Epoch 6/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 813us/step - accuracy: 0.7289 - loss: 0.5526
Epoch 7/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 848us/step - accuracy: 0.7350 - loss: 0.5440
Epoch 8/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 971us/step - accuracy: 0.7318 - loss: 0.5470
Epoch 9/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 849us/step - accuracy: 0.7362 - loss: 0.5396
Epoch 10/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 838us/step - accuracy: 0.7306 - loss: 0.5487
Epoch 11/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 899us/step - accuracy: 0.7366 - loss: 0.5404
Epoch 12/50
912/912 ━━━━━━━━━━

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

161/161 - 0s - 2ms/step - accuracy: 0.7273 - loss: 0.5550
Loss: 0.5550448894500732, Accuracy: 0.7273080945014954


# Second Attempt at Optimization

In [76]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
layer1 = 40
layer2 = 20
layer3 = 10
layer4 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1, activation="leaky_relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, activation="leaky_relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=layer3, activation="leaky_relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=layer4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_61 (Dense)                │ (None, 40)             │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 20)             │           820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 5)              │            55 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,691 (14.42 KB)

 Trainable params: 3,691 (14.42 KB)

 Non-trainable params: 0 (0.00 B)

In [77]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [78]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7058 - loss: 0.5888
Epoch 2/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 959us/step - accuracy: 0.7295 - loss: 0.5544
Epoch 3/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7342 - loss: 0.5461
Epoch 4/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7332 - loss: 0.5474
Epoch 5/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 999us/step - accuracy: 0.7371 - loss: 0.5421
Epoch 6/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7332 - loss: 0.5449
Epoch 7/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 985us/step - accuracy: 0.7338 - loss: 0.5468
Epoch 8/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 962us/step - accuracy: 0.7341 - loss: 0.5458
Epoch 9/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 977us/step - accuracy: 0.7354 - loss: 0.5419
Epoch 10/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7310 - loss: 0.5471
Epoch 11/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7368 - loss: 0.5418
Epoch 12/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1

In [79]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

161/161 - 0s - 1ms/step - accuracy: 0.7300 - loss: 0.5539
Loss: 0.553851306438446, Accuracy: 0.7300291657447815


In [80]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5", save_format='h5')

# Third Attempt at Optimization

In [72]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
layer1 = 50
layer2 = 20
layer3 = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1, activation="leaky_relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, activation="relu"))

# Third  hidden layer
nn.add(tf.keras.layers.Dense(units=layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

c:\Users\rmt20\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_57 (Dense)                │ (None, 50)             │         3,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_58 (Dense)                │ (None, 20)             │         1,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_59 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,491 (17.54 KB)

 Trainable params: 4,491 (17.54 KB)

 Non-trainable params: 0 (0.00 B)

In [73]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [74]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 2s 919us/step - accuracy: 0.7141 - loss: 0.5905
Epoch 2/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 997us/step - accuracy: 0.7303 - loss: 0.5528
Epoch 3/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 882us/step - accuracy: 0.7342 - loss: 0.5468
Epoch 4/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 956us/step - accuracy: 0.7289 - loss: 0.5510
Epoch 5/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 892us/step - accuracy: 0.7354 - loss: 0.5474
Epoch 6/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 903us/step - accuracy: 0.7330 - loss: 0.5445
Epoch 7/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 874us/step - accuracy: 0.7352 - loss: 0.5447
Epoch 8/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 963us/step - accuracy: 0.7370 - loss: 0.5379
Epoch 9/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 906us/step - accuracy: 0.7348 - loss: 0.5421
Epoch 10/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 943us/step - accuracy: 0.7357 - loss: 0.5444
Epoch 11/50
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 875us/step - accuracy: 0.7321 - loss: 0.5455
Epoch 12/50
912/912 ━━━━━━━━━━

In [75]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

161/161 - 0s - 1ms/step - accuracy: 0.7287 - loss: 0.5514
Loss: 0.5514083504676819, Accuracy: 0.7286686301231384
